# MovieLens 数据分析（SQL in Python 环境）

#### 除了第一个任务 剩下任务仅显示前20行，表格其余信息另存在相应的csv文件中

In [21]:
import pandas as pd
import sqlite3
# # 设置 pandas 显示所有行
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)


In [22]:
# 加载数据
movies_df = pd.read_csv("movies.csv")
ratings_df = pd.read_csv("ratings.csv")

# 创建内存数据库
conn = sqlite3.connect(":memory:")

# 写入数据库
movies_df.to_sql("movies", conn, index=False, if_exists="replace")
ratings_df.to_sql("ratings", conn, index=False, if_exists="replace")


105339

## 任务一：平均得分前10的电影

In [23]:
query1 = '''
SELECT 
    m.title,
    AVG(r.rating) AS avg_rating
FROM 
    ratings r
JOIN 
    movies m ON r.movieId = m.movieId
GROUP BY 
    m.title
ORDER BY 
    avg_rating DESC
LIMIT 10;
'''
pd.read_sql_query(query1, conn)


,title,avg_rating
0,Young at Heart (a.k.a. Young@Heart) (2007),5.0
1,"Women on the 6th Floor, The (Les Femmes du 6èm...",5.0
2,Wings (1927),5.0
3,Werckmeister Harmonies (Werckmeister harmóniák...,5.0
4,War Photographer (2001),5.0
5,Waiting for 'Superman' (2010),5.0
6,"Traviata, La (1982)",5.0
7,Topkapi (1964),5.0
8,Time of the Gypsies (Dom za vesanje) (1989),5.0
9,Three Ages (1923),5.0


## 任务二：每个类型的平均得分前10的电影（Python 拆分类型）

In [31]:
merged_df = pd.merge(ratings_df, movies_df, on="movieId")
merged_df['genres'] = merged_df['genres'].str.split('|')
genre_df = merged_df.explode('genres')

genre_df.to_sql("genre_expanded", conn, index=False, if_exists="replace")

query2 = '''
WITH genre_avg AS (
    SELECT 
        genres AS genre,
        title,
        AVG(rating) AS avg_rating
    FROM 
        genre_expanded
    GROUP BY 
        genre, title
),
ranked AS (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY genre ORDER BY avg_rating DESC) AS rank
    FROM genre_avg
)
SELECT genre, title, avg_rating
FROM ranked
WHERE rank <= 10;
'''

task2=pd.read_sql_query(query2, conn)

# 显示前 20 行
display(task2.head(20))

# 导出完整结果为 CSV 文件
task2.to_csv("task2_top10_movies_per_genre.csv", index=False)

,genre,title,avg_rating
0,(no genres listed),Marco Polo: One Hundred Eyes (2015),4.0
1,(no genres listed),Round Trip to Heaven (1992),4.0
2,(no genres listed),Pablo (2012),3.5
3,(no genres listed),The Take (2009),3.5
4,(no genres listed),The 50 Year Argument (2014),2.5
5,(no genres listed),Li'l Quinquin ( ),2.0
6,(no genres listed),The Big Broadcast of 1936 (1935),2.0
7,Action,"Chase, The (1994)",5.0
8,Action,"Friend Is a Treasure, A (Chi Trova Un Amico, T...",5.0
9,Action,Ghost in the Shell: Stand Alone Complex - The ...,5.0


## 任务三：每个用户评分最高的前5类型

In [32]:
query3 = '''
WITH genre_avg AS (
    SELECT 
        userId,
        genres AS genre,
        AVG(rating) AS avg_rating
    FROM 
        genre_expanded
    GROUP BY 
        userId, genre
),
ranked AS (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY userId ORDER BY avg_rating DESC) AS rank
    FROM genre_avg
)
SELECT userId, genre, avg_rating
FROM ranked
WHERE rank <= 5
'''
task3 = pd.read_sql_query(query3, conn)

display(task3.head(20))
task3.to_csv("task3_top5_genre_user1.csv", index=False)

,userId,genre,avg_rating
0,1,Crime,4.209677
1,1,War,4.200000
2,1,Animation,4.000000
3,1,Film-Noir,4.000000
4,1,Musical,4.000000
5,2,Animation,4.500000
6,2,Drama,4.363636
7,2,Children,4.333333
8,2,Crime,4.333333
9,2,Fantasy,4.250000


### 任务三：每个用户评分最高的前5类型(其他表现新形势)

In [33]:
top5_df = pd.read_sql_query(query3, conn)

# 转换为透视表形式：userId 为行，genre 为列，值为 avg_rating
pivot_table = top5_df.pivot(index='userId', columns='genre', values='avg_rating')

# Nah填入‘-’
pivot_table = pivot_table.fillna('-')

display(pivot_table.head(20))
pivot_table.to_csv("task3_top5_genre_user2.csv", index=False)

genre,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
userId,,,,,,,,,,,,,,,,,,,,
1,-,-,-,4.0,-,-,4.209677,-,-,-,4.0,-,-,4.0,-,-,-,-,4.2,-
2,-,-,-,4.5,4.333333,-,4.333333,-,4.363636,4.25,-,-,-,-,-,-,-,-,-,-
3,-,-,-,-,-,-,4.0,5.0,-,-,-,4.0,4.0,-,4.25,-,-,-,-,-
4,-,-,-,4.75,4.333333,-,-,-,4.342105,-,-,-,-,-,4.4,-,-,-,4.5625,-
5,-,-,-,4.095238,3.904762,-,-,-,-,3.84375,-,-,4.277778,4.090909,-,-,-,-,-,-
6,-,-,-,-,-,-,4.363636,-,-,-,-,4.5,-,4.5,-,4.366667,4.333333,-,-,-
7,-,-,-,-,-,-,3.854839,4.0,-,-,5.0,-,-,5.0,3.861111,-,-,-,-,-
8,-,-,-,4.333333,4.166667,-,-,-,4.133333,4.0,-,-,-,-,4.333333,-,-,-,-,-
9,-,-,-,-,-,-,3.086957,3.0,-,-,-,-,4.0,-,-,-,-,2.866667,3.0,-


## 任务四：每个用户观影次数最多的前5类型

In [34]:
query4 = '''
WITH genre_counts AS (
    SELECT 
        userId,
        genres AS genre,
        COUNT(*) AS view_count
    FROM 
        genre_expanded
    GROUP BY 
        userId, genre
),
ranked AS (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY userId ORDER BY view_count DESC) AS rank
    FROM genre_counts
)
SELECT userId, genre, view_count
FROM ranked
WHERE rank <= 5
'''
task4 = pd.read_sql_query(query4, conn)

display(task4.head(20))
task4.to_csv("task4_top5_viewcount_user1.csv", index=False)

,userId,genre,view_count
0,1,Action,46
1,1,Drama,45
2,1,Thriller,43
3,1,Adventure,31
4,1,Comedy,31
5,2,Thriller,12
6,2,Comedy,11
7,2,Drama,11
8,2,Adventure,10
9,2,Action,9


In [35]:
top5_vc = pd.read_sql_query(query4, conn)
pivot_table = top5_vc.pivot(index='userId', columns='genre', values='view_count')
pivot_table = pivot_table.fillna('-')

display(pivot_table.head(20))
pivot_table.to_csv("task4_top5_viewcount_user12.csv", index=False)

genre,Action,Adventure,Animation,Children,Comedy,Crime,Drama,Fantasy,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War
userId,,,,,,,,,,,,,,,,
1,46.0,31.0,-,-,31.0,-,45.0,-,-,-,-,-,-,-,43.0,-
2,9.0,10.0,-,-,11.0,-,11.0,-,-,-,-,-,-,-,12.0,-
3,13.0,-,-,-,35.0,-,36.0,-,-,-,-,-,22.0,-,21.0,-
4,-,-,-,-,46.0,18.0,76.0,-,-,-,-,-,37.0,-,18.0,-
5,-,22.0,21.0,21.0,45.0,-,-,-,-,-,-,-,21.0,-,-,-
6,-,14.0,-,-,28.0,-,31.0,-,-,-,-,-,15.0,-,13.0,-
7,95.0,53.0,-,-,46.0,-,-,-,-,-,-,-,-,63.0,71.0,-
8,-,12.0,-,-,24.0,-,30.0,-,-,-,-,-,17.0,-,16.0,-
9,39.0,-,-,-,53.0,-,49.0,-,-,-,-,-,24.0,-,45.0,-
